<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_proc_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script 1: procesamiento NLU

##  Entorno

In [1]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json
import re

In [ ]:
# procesar con GPU

In [2]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [3]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [4]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


Lectura de tablas de datos primarios

In [5]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

Instalar librería STANZA (Stanford NLP)

In [ ]:
! pip install stanza

In [7]:
import stanza
stanza.download('es') # download spanish model

2020-11-02 20:17:37 INFO: Downloading default packages for language: es (Spanish)...
2020-11-02 20:27:40 INFO: Finished downloading models and saved to /root/stanza_resources.


# Prueba de Tokenizacion de un sumario

In [8]:
fallos = tbfallos.loc[1:100, ['textos_fallo']] # selecting row and cols

In [9]:
fallos['textos_fallo'] = fallos.textos_fallo.str.replace("#","\n\n") # or fallos.textos_fallo.replace("#","\n\n", regex=True) 

In [10]:
corpus = fallos.textos_fallo.to_list()

In [ ]:
corpus[1]

In [24]:
len(corpus)

100

In [23]:
type(corpus)

list

In [13]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma', tokenize_no_ssplit=True)

2020-11-02 20:27:59 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

2020-11-02 20:27:59 INFO: Use device: gpu
2020-11-02 20:27:59 INFO: Loading: tokenize
2020-11-02 20:28:08 INFO: Loading: mwt
2020-11-02 20:28:08 INFO: Loading: pos
2020-11-02 20:28:09 INFO: Loading: lemma
2020-11-02 20:28:09 INFO: Done loading processors!


Pruebas para procesamiento en lotes para extracción de lemas y otros datos

In [14]:
def print_word_info(word):
    print(f"Text:\t{word.text}")
    print(f"Lemma: \t{word.lemma}")
    print(f"UPOS: \t{word.upos}")
    print(f"XPOS: \t{word.xpos}")

def word_info_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame object with one row for each token in
      doc, and columns for text, lemma, upos, and xpos.
    """
    rows = []
    for sentence in doc.sentences:
        for word in sentence.words:
            row = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
            }
            rows.append(row)
    return pd.DataFrame(rows)



In [15]:
df = pd.DataFrame()
counter = 0

for text in corpus:
  counter = counter + 1 
  temp = nlp(text)
  temp = word_info_df(temp)
  temp["fallo_nro"] = counter
  df = pd.concat([df, temp], axis=0)  

In [16]:
df

,text,lemma,upos,xpos,fallo_nro
0,TEXTO,texto,NOUN,NOUN,1
1,COMPLETO,completo,ADJ,ADJ,1
2,A,a,ADP,ADP,1
3,C,c,NOUN,NOUN,1
4,U,U,CCONJ,CCONJ,1
...,...,...,...,...,...
6905,SORIA,SORIA,PROPN,PROPN,100
6906,JUAN,JUAN,PROPN,PROPN,100
6907,JOSE,JOSE,PROPN,PROPN,100
6908,MARTIARENA,MARTIARENA,PROPN,PROPN,100


In [17]:
df.lemma.unique()

array(['texto', 'completo', 'a', ..., 'reembolsar', 'invalidado',
       '118176'], dtype=object)

In [30]:
def proc_lemas(lista_raw_docs):
  """
  - Parameters: list of raw documents, sep: newline(n)newline(n)
  - Returns: a pd df columns for text, lemma, upos, and xpos.
  """
  df = pd.DataFrame()
  counter = 0
  nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma', tokenize_no_ssplit=True)

  for text in corpus:
    counter = counter + 1 
    temp = nlp(text)
    temp = word_info_df(temp)
    temp["fallo_nro"] = counter
    df = pd.concat([df, temp], axis=0)  
  return df

In [ ]:
df_lemas = proc_lemas(corpus)

In [33]:
df_lemas.lemma.unique()

array(['texto', 'completo', 'a', ..., 'reembolsar', 'invalidado',
       '118176'], dtype=object)